# Importing

In [ ]:
import pdfplumber
import os
import csv
import pandas as pd
import re
import ctypes
from ctypes.util import find_library
import camelot

# Create a txt file with all data

In [ ]:
filepath = "C:/Users/bsavoiumarinas/Documents/Tesi/PyProjects/Doc_Panthera/"
texts_pages = []
text = ""

# In data extract each folder one by one
for folder in os.listdir(filepath):
    folder_path = os.path.join(filepath, folder)
    
    # Check if the folder_path is actually a directory
    if os.path.isdir(folder_path):
        # For each file in the folder, extract the file name
        for filename in os.listdir(folder_path):
            if filename.endswith(".pdf"):
                filename_path = os.path.join(folder_path, filename)
                
                # Use 'with' to automatically close the file after reading
                with pdfplumber.open(filename_path) as pdf:
                    # Extract text from each page with error handling for None
                    texts_pages = [page.extract_text(x_tolerance=1) or "" for page in pdf.pages]
                    # Concatenate all the text together in a single string
                    text += "  \n\n".join(texts_pages)
 

In [ ]:
# Write on file the text string
filename_all_data = "./Files/all_data.txt"

if not os.path.exists(filename_all_data):
  with open(filename_all_data, "w", encoding="utf-8") as f:
    f.write(text)

In [ ]:
print(text)

In [ ]:
print(len(text))

In [ ]:
file = open(filename_all_data, "r", encoding="utf-8")
string_data = ""

for line in file.readlines():
    string_data += line
file.close()

In [ ]:
print(len(string_data))

In [ ]:
print(string_data)

# Trial for only a folder

In [ ]:
filepath = "./Doc_Panthera/Fast update"
# Create a dictionary called texts
texts_dict = {}

for filename in os.listdir(filepath):
  if filename.endswith(".pdf"):
    filename_path = os.path.join(filepath, filename)
    pdf = pdfplumber.open(filename_path)   
    
    pages_text_for_file = [page.extract_text(x_tolerance=1) for page in pdf.pages]
    # Can concatenate all the text together in a single string
    text_for_file = "  \n\n".join(pages_text_for_file)
      
    # Populate the texts dictionary, having as key the name of the document and as value the string of all the text
    texts_dict[filename] = text_for_file
  

In [ ]:
print(texts_dict.keys())

In [ ]:
print(texts_dict['Fastupdate A&C 5_0_03.pdf'])

In [ ]:
filename_all_data_trial = "./Files/trial_dict.csv"
with open(filename_all_data_trial, 'w', newline='', encoding = 'utf-8') as csvfile:
    writer = csv.writer(csvfile)
    for key, value in texts_dict.items():
        writer.writerow([key, value])

In [ ]:
df = pd.read_csv(filename_all_data_trial, names = ['file', 'text'], header = None)

In [ ]:
df

# Create a csv file with all data in a dictionary

In [ ]:
filepath = "./Doc_Panthera/"
# Create a dictionary called texts
texts_dict = {}

# In data extract each folder one by one
for folder in os.listdir(filepath):
  # For each file in the filepath extract the file name
  folder_path = os.path.join(filepath, folder)
  for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
      filename_path = os.path.join(folder_path, filename)
      pdf = pdfplumber.open(filename_path)   
    
      pages_text_for_file = [page.extract_text(x_tolerance=1) for page in pdf.pages]
      # Can concatenate all the text together in a single string
      text_for_file = "  \n\n".join(pages_text_for_file)
      
      # Populate the texts dictionary, having as key the name of the document and as value the string of all the text
      texts_dict[filename] = text_for_file

In [ ]:
filename_all_data_dict = "./Files/all_data_dict.csv"
with open(filename_all_data_dict, 'w', newline='', encoding = 'utf-8') as csvfile:
    writer = csv.writer(csvfile)
    for key, value in texts_dict.items():
        writer.writerow([key, value])

In [ ]:
data_df = pd.read_csv(filename_all_data_dict, names = ['file', 'text'], header = None)

In [ ]:
data_df

# Create the csv file with a dictionary {file_name: text} importing directly the pdf text without headers and footers

Reference: https://pypdf2.readthedocs.io/en/3.0.0/user/extract-text.html#example-1-ignore-header-and-footer

In [ ]:
# Exclude headers and footer directly when importing the pdf 
# Use the y-coordinates to extract text on the page
def extract_text_by_y_coordinate(page, min_y=50, max_y=750, parts=None):
    """
    Extracts text from a page based on Y-coordinate range and appends it to parts.

    Args:
    - page (pdfplumber.page.Page): The page object to extract text from.
    - min_y (float): Minimum Y-coordinate for filtering text.
    - max_y (float): Maximum Y-coordinate for filtering text.
    - parts (list): List to append filtered text parts to.
    """
    if parts is None:
        parts = []

    for char in page.chars:
        y = char['top']
        if min_y < y < max_y:
            parts.append(char['text'])

    return parts

def extract_text_from_all_pages(file_path, min_y=50, max_y=750):
    """
    Extracts text from all pages of a PDF file based on Y-coordinate range. We take Y-coordinates between 50 and 720 to exclude the heading and footing of the page, given that a typical PDF document may have a page height of 792 units. The ranges we use ar the ones suggested by the pdfplumber documentation.

    Args:
    - file_path (str): Path to the PDF file.
    - min_y (float): Minimum Y-coordinate for filtering text.
    - max_y (float): Maximum Y-coordinate for filtering text.

    Returns:
    - str: Extracted text from all pages and coordinate range.
    """
    parts = []

    with pdfplumber.open(file_path) as pdf:
        parts = [extract_text_by_y_coordinate(page, min_y, max_y, parts) for page in pdf.pages]
        # Can concatenate all the text together in a single string
        text_for_file = "  \n\n".join(pages_text_for_file)
        #parts = extract_text_by_y_coordinate(page, min_y, max_y, parts)

    return "".join(parts)

In [ ]:
# Define your coordinate-based text extraction function
def extract_text_by_coordinate(page, min_y=50, max_y=750):
    """
    Extracts text from a page based on Y-coordinate range and returns a string.

    Args:
    - page (pdfplumber.page.Page): The page object to extract text from.
    - min_y (float): Minimum Y-coordinate for filtering text.
    - max_y (float): Maximum Y-coordinate for filtering text.

    Returns:
    - str: Extracted text from the page.
    """
    text_lines = []
    last_y = None

    for char in page.chars:
        y = char['top']
        text = char['text']
        
        if min_y < y < max_y:
            if last_y is not None and abs(y - last_y) > 10:  # Adjust threshold for line breaks
                text_lines.append('\n')  # Insert a new line for vertical gaps
            text_lines.append(text)
        
        last_y = y

    # Join text lines into a single string and return
    return "".join(text_lines)

In [ ]:
# Directory containing PDF files
filepath = "./Doc_Panthera/"

# Create a dictionary to store text from PDFs
texts_dict = {}

# Loop through each folder in the specified filepath
for folder in os.listdir(filepath):
    # Get the folder path
    folder_path = os.path.join(filepath, folder)
    
    # Loop through files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            filename_path = os.path.join(folder_path, filename)
            with pdfplumber.open(filename_path) as pdf:
                # Extract text from each page and combine
                parts = [extract_text_by_coordinate(page) for page in pdf.pages]
                text_for_file = "\n\n".join(parts)  # Combine text from all pages

            # Add to dictionary, using the filename as the key
            texts_dict[filename] = text_for_file

In [ ]:
filename_all_data_dict = "./Files/data_imported_by_pdf_coordinates.csv"
with open(filename_all_data_dict, 'w', newline='', encoding = 'utf-8') as csvfile:
    writer = csv.writer(csvfile)
    for key, value in texts_dict.items():
        writer.writerow([key, value])

In [ ]:
data_df_only_text = pd.read_csv(filename_all_data_dict, names = ['file', 'text'], header = None)

In [ ]:
data_df_only_text

In [ ]:
# Create a list with all the values in the column 'file'
file_list = data_df_only_text['file'].tolist()

# Create a list with all the values in the column 'text'
text_list = data_df_only_text['text'].tolist()

In [ ]:
text_list[56]

# Complementar import - check not imported page elements to tune the 'y' coordinate

PDF headers

In [ ]:
# Directory containing PDF files
filepath = "./Doc_Panthera/"

# Create a dictionary to store text from PDFs
texts_dict = {}

# Loop through each folder in the specified filepath
for folder in os.listdir(filepath):
    # Get the folder path
    folder_path = os.path.join(filepath, folder)
    
    # Loop through files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            filename_path = os.path.join(folder_path, filename)
            with pdfplumber.open(filename_path) as pdf:
                # Extract text from each page and combine
                parts = [extract_text_by_coordinate(page, 0, 50) for page in pdf.pages]
                text_for_file = "\n\n".join(parts)  # Combine text from all pages

            # Add to dictionary, using the filename as the key
            texts_dict[filename] = text_for_file

In [ ]:
filename_all_data_dict = "./Files/headings.csv"
with open(filename_all_data_dict, 'w', newline='', encoding = 'utf-8') as csvfile:
    writer = csv.writer(csvfile)
    for key, value in texts_dict.items():
        writer.writerow([key, value])

In [ ]:
filename_all_data_dict = "./Files/headings.csv"
data_df_only_text = pd.read_csv(filename_all_data_dict, names = ['file', 'text'], header = None)

In [ ]:
data_df_only_text[data_df_only_text['text'].str.contains(r'\w')]

In [ ]:
# Create a list with all the values in the column 'text'
text_list = data_df_only_text['text'].tolist()

In [ ]:
# Create a list with all the values in the column 'text'
import re
def clean_output_print(text):
    pattern = r"\n"
    
    cleaned_text = re.sub(pattern, '', text, flags=re.MULTILINE)
    return cleaned_text

text_list = data_df_only_text['text'].tolist()
for i in range(len(text_list)):
    text_list[i] = clean_output_print(text_list[i])

text_list

PDF footers

In [ ]:
# Directory containing PDF files
filepath = "./Doc_Panthera/"

# Create a dictionary to store text from PDFs
texts_dict = {}

# Loop through each folder in the specified filepath
for folder in os.listdir(filepath):
    # Get the folder path
    folder_path = os.path.join(filepath, folder)
    
    # Loop through files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            filename_path = os.path.join(folder_path, filename)
            with pdfplumber.open(filename_path) as pdf:
                # Extract text from each page and combine
                parts = [extract_text_by_coordinate(page, 750, 800) for page in pdf.pages]
                text_for_file = "\n\n".join(parts)  # Combine text from all pages

            # Add to dictionary, using the filename as the key
            texts_dict[filename] = text_for_file

In [ ]:
filename_all_data_dict = "./Files/footing.csv"
with open(filename_all_data_dict, 'w', newline='', encoding = 'utf-8') as csvfile:
    writer = csv.writer(csvfile)
    for key, value in texts_dict.items():
        writer.writerow([key, value])

In [ ]:
filename_all_data_dict = "./Files/footing.csv"
data_df_only_text = pd.read_csv(filename_all_data_dict, names = ['file', 'text'], header = None)

In [ ]:
regex_letter = r"\w"
data_df_only_text[data_df_only_text['text'].str.contains(regex_letter)]

In [ ]:
def clean_output_print(text):
    pattern = r"\n"
    
    cleaned_text = re.sub(pattern, '', text, flags=re.MULTILINE)
    return cleaned_text

In [ ]:
# Create a list with all the values in the column 'text'
text_list = data_df_only_text['text'].tolist()
for i in range(len(text_list)):
    text_list[i] = clean_output_print(text_list[i])


In [ ]:
print(text_list)

In [ ]:
path = "./Caricamenti massa/CaricamentMassa_Excel.pdf"
with pdfplumber.open(path) as pdf:
    text = [extract_text_by_coordinate(page, 50, 750) for page in pdf.pages]
    text_for_file = "\n\n".join(text)  # Combine text from all pages
    
text_for_file

# Extract the tables from the files

In [ ]:
find_library(r"C:\Program Files\gs\gs10.04.0\bin\gsdll64.dll")

In [ ]:
path = "./Documentazione tecnica/GEN_Runtimetools.pdf"
tables = camelot.read_pdf(path, "13")
tables

In [ ]:
tables[0].df

In [ ]:
def extract_tables_from_pdfs(input_folder, output_folder):
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Loop over all PDF files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(input_folder, filename)
            print(f"Processing: {pdf_path}")
            
            # Extract tables from all pages of the PDF
            tables = camelot.read_pdf(pdf_path, pages="all")
            
            # Create an output subfolder for this PDF file (using filename without extension)
            pdf_output_folder = os.path.join(output_folder, os.path.splitext(filename)[0])
            if not os.path.exists(pdf_output_folder):
                os.makedirs(pdf_output_folder)
            
            # Loop through all extracted tables
            for i, table in enumerate(tables):
                df = table.df
                
                # Remove rows with all NaN or empty values
                df_cleaned = df.replace("", pd.NA).dropna(how='all')
                
                # Skip the table if it's empty after cleaning (no non-null rows)
                if not df_cleaned.empty:
                    # Get page number for the current table
                    page_num = table.page
                    
                    # Define the CSV path (use page number and index to avoid overwriting)
                    csv_filename = f"table_page_{page_num}_table_{i+1}.csv"
                    csv_path = os.path.join(pdf_output_folder, csv_filename)
                    
                    # Save the cleaned table to a CSV file
                    df_cleaned.to_csv(csv_path, index=False)
                    print(f"Saved table {i+1} from page {page_num} to {csv_path}")
                else:
                    print(f"Table {i+1} on page {table.page} in {filename} is empty, skipping.")

input_folder = "./Doc_Panthera/Fast update"
output_folder = "./Experiments/Tables"

extract_tables_from_pdfs(input_folder, output_folder)


In [ ]:
def find_and_remove_empty_folders(directory):
    # Iterate over all subdirectories and files
    for root, dirs, files in os.walk(directory, topdown=False):  # bottom-up to ensure empty parent directories can also be removed
        for folder in dirs:
            folder_path = os.path.join(root, folder)
            if not os.listdir(folder_path):  # Check if the folder is empty
                os.rmdir(folder_path)  # Remove the empty folder
                print(f"Removed empty folder: {folder_path}")

# Example usage:
# Replace 'your_directory_path' with the path of the folder you want to check
find_and_remove_empty_folders('./Experiments/Tables')
